# {mod}`ultralytics` 测试样例

In [1]:
from tasks import set_env
from pathlib import Path
temp_dir = Path("./.temp") # 设置缓存目录
(temp_dir/"cfg/models").mkdir(exist_ok=True, parents=True)

In [2]:
%%file {temp_dir}/cfg/models/yolo11-custom.yaml

# Ultralytics YOLO 🚀，采用 AGPL-3.0许可
# YOLOv11 目标检测模型，支持 P3-P5 的输出。使用示例请参阅 https://docs.ultralytics.com/tasks/detect

# 参数
nc: 80 # n类别数量
scales: # 模型复合缩放常数，即 `'model=yolo11n.yaml'` 会调用带有缩放因子 `'n'` 的 yolo11.yaml。
  # [depth, width, max_channels]
  n: [0.50, 0.25, 1024] # summary: 319 layers, 2624080 parameters, 2624064 gradients, 6.6 GFLOPs
  s: [0.50, 0.50, 1024] # summary: 319 layers, 9458752 parameters, 9458736 gradients, 21.7 GFLOPs
  m: [0.50, 1.00, 512] # summary: 409 layers, 20114688 parameters, 20114672 gradients, 68.5 GFLOPs
  l: [1.00, 1.00, 512] # summary: 631 layers, 25372160 parameters, 25372144 gradients, 87.6 GFLOPs
  x: [1.00, 1.50, 512] # summary: 631 layers, 56966176 parameters, 56966160 gradients, 196.0 GFLOPs

# YOLO11n backbone
backbone:
  # [from, repeats, module, args]
  - [-1, 1, Conv, [64, 3, 2]] # 0-P1/2
#   - [-1, 1, Conv, [128, 3, 2]] # 1-P2/4
#   - [-1, 2, C3k2, [256, False, 0.25]]
#   - [-1, 1, Conv, [256, 3, 2]] # 3-P3/8
#   - [-1, 2, C3k2, [512, False, 0.25]]
#   - [-1, 1, Conv, [512, 3, 2]] # 5-P4/16
#   - [-1, 2, C3k2, [512, True]]
#   - [-1, 1, Conv, [1024, 3, 2]] # 7-P5/32
#   - [-1, 2, C3k2, [1024, True]]
#   - [-1, 1, SPPF, [1024, 5]] # 9
#   - [-1, 2, C2PSA, [1024]] # 10

# YOLO11n head
head:
  - [-1, 1, nn.Upsample, [None, 2, "nearest"]]
#   - [[-1, 6], 1, Concat, [1]] # cat backbone P4
#   - [-1, 2, C3k2, [512, False]] # 13

#   - [-1, 1, nn.Upsample, [None, 2, "nearest"]]
#   - [[-1, 4], 1, Concat, [1]] # cat backbone P3
#   - [-1, 2, C3k2, [256, False]] # 16 (P3/8-small)

#   - [-1, 1, Conv, [256, 3, 2]]
#   - [[-1, 13], 1, Concat, [1]] # cat head P4
#   - [-1, 2, C3k2, [512, False]] # 19 (P4/16-medium)

#   - [-1, 1, Conv, [512, 3, 2]]
#   - [[-1, 10], 1, Concat, [1]] # cat head P5
#   - [-1, 2, C3k2, [1024, True]] # 22 (P5/32-large)

#   - [[16, 19, 22], 1, Detect, [nc]] # Detect(P3, P4, P5)

Overwriting .temp/cfg/models/yolo11-custom.yaml


In [3]:
from ultralytics import YOLO
# 加载模型
model = YOLO(f"{temp_dir}/cfg/models/yolo11n-custom.yaml", task="detect")  # 利用 YAML 构建新模型
# 训练模型
# results = model.train(data="coco8.yaml", epochs=100, imgsz=640)

In [4]:
path = model.export(format="onnx")  # 返回导出模型的路径

Ultralytics 8.3.28 🚀 Python-3.12.2 torch-2.5.0 CPU (Intel Xeon E5-2678 v3 2.50GHz)

PyTorch: starting from 'yolo11n-custom.yaml' with input shape (1, 3, 640, 640) BCHW and output shape(s) (1, 16, 640, 640) (0.0 MB)

ONNX: starting export with onnx 1.16.2 opset 19...
ONNX: slimming with onnxslim 0.1.34...
ONNX: export success ✅ 0.4s, saved as 'yolo11n-custom.onnx' (0.0 MB)

Export complete (0.4s)
Results saved to /media/pc/data/lxw/ai/torch-book/doc/ultralytics
Predict:         yolo predict task=detect model=yolo11n-custom.onnx imgsz=640  
Validate:        yolo val task=detect model=yolo11n-custom.onnx imgsz=640 data=None  
Visualize:       https://netron.app
